# Predictive Modeling: Italy Energy Consumption Forecast (10-Year Span)

This notebook performs machine learning on energy data for Italy, building predictive models to forecast energy consumption for the next 10 years.

## 1. Import Required Libraries

In [ ]:
using Pkg
Pkg.add(["CSV", "DataFrames", "Downloads", "Statistics", "Plots", "MLJ", "TimeSeries", "StatsPlots", "Dates"])

using CSV, DataFrames, Downloads, Statistics, Plots, MLJ, TimeSeries, StatsPlots, Dates
println("All libraries imported successfully!")

## 2. Load and Explore Italian Energy Data

In [ ]:
# Download global energy dataset
url = "https://raw.githubusercontent.com/owid/energy-data/master/owid-energy-data.csv"
filename = "owid-energy-data.csv"

# Download if not already present
if !isfile(filename)
    println("Downloading energy data...")
    Downloads.download(url, filename)
else
    println("Energy data file already exists")
end

# Load data
println("Loading energy data...")
energy_data = CSV.read(filename, DataFrame)

# Filter for Italy
italy_data = filter(row -> row.country == "Italy", energy_data)

# Sort by year
sort!(italy_data, :year)

println("Italy Energy Data Shape: $(size(italy_data))")
println("\nFirst few rows:")
first(italy_data, 5)

# Check available columns
println("\nColumn names:")
println(names(italy_data))

## 3. Data Preprocessing and Feature Engineering

In [ ]:
# Focus on primary energy consumption column
# Check which energy columns are available
energy_cols = [col for col in names(italy_data) if contains(lowercase(col), "energy") || contains(lowercase(col), "consumption")]
println("Available energy columns: $energy_cols")

# Use primary energy consumption (adjust column name based on available data)
target_col = :primary_energy_consumption

# Handle missing values
italy_clean = dropmissing(italy_data[:, [:year, target_col]])

println("\nData after removing missing values: $(size(italy_clean))")
println("Year range: $(minimum(italy_clean.year)) - $(maximum(italy_clean.year))")

# Display summary statistics
println("\nTarget Variable Summary Statistics:")
println("Mean: $(mean(italy_clean[!, target_col]))")
println("Std: $(std(italy_clean[!, target_col]))")
println("Min: $(minimum(italy_clean[!, target_col]))")
println("Max: $(maximum(italy_clean[!, target_col]))")
println("\nFirst few rows after preprocessing:")
first(italy_clean, 5)

## 4. Prepare Time Series Data for Modeling

In [ ]:
# Extract year and energy values
years = italy_clean.year
energy_values = italy_clean[!, target_col]

# Normalize the data for better model performance
mean_val = mean(energy_values)
std_val = std(energy_values)
X_normalized = (energy_values .- mean_val) ./ std_val

# Create year index starting from 0
X_time = collect(0:(length(years)-1))

# Combine into a matrix for MLJ models
X = reshape(X_time, :, 1)  # Independent variable: relative time
y = energy_values           # Dependent variable: energy consumption

println("Training data shape: $(size(X))")
println("Target data shape: $(size(y))")
println("Years covered: $(length(years)) years")
println("\nData prepared for modeling!")

## 5. Train Predictive Models

In [ ]:
using MLJ

# Define models
models = Dict(
    "Linear Regressor" => @load LinearRegressor pkg=MLJLinearModels,
    "Ridge Regressor" => @load RidgeRegressor pkg=MLJLinearModels,
    "Gaussian Process" => @load GaussianProcessRegressor pkg=MLJGaussianProcesses
)

# Train models and evaluate
results = Dict()

println("Training models...\n")

for (name, model_type) in models
    try
        println("Training: $name")
        model = model_type()
        
        # Convert to MLJ-compatible format
        X_table = MLJ.table(X)
        
        # Train on full dataset
        mach = machine(model, X_table, y)
        fit!(mach, verbosity=0)
        
        # Get predictions on training data
        y_pred = predict(mach, X_table)
        
        # Calculate metrics
        mae = mean(abs.(y_pred .- y))
        rmse = sqrt(mean((y_pred .- y).^2))
        
        results[name] = (model=mach, mae=mae, rmse=rmse)
        
        println("  MAE: $(round(mae, digits=2))")
        println("  RMSE: $(round(rmse, digits=2))")
        println()
        
    catch e
        println("  Error training $name: $(e)")
        println()
    end
end

println("Model training completed!")

## 6. Generate 10-Year Forecasts

In [ ]:
# Use the best model (Linear Regressor) for forecasting
best_model_name = first(results).first  # Get first model
best_model = results[best_model_name][:model]

# Generate future years (next 10 years)
last_year = maximum(years)
future_years = (last_year + 1):(last_year + 10)
future_X_time = collect(length(years):(length(years) + 9))

# Create prediction matrix for future years
X_future = reshape(future_X_time, :, 1)
X_future_table = MLJ.table(X_future)

# Make predictions for the next 10 years
try
    forecast_values = predict(best_model, X_future_table)
    
    # Combine historical and forecast data
    all_years = vcat(years, future_years)
    all_values = vcat(energy_values, forecast_values)
    
    # Create a dataframe for easy viewing
    forecast_df = DataFrame(
        Year = future_years,
        Forecast = forecast_values
    )
    
    println("10-Year Forecast for Italy (Primary Energy Consumption):")
    println(forecast_df)
    
catch e
    println("Error generating forecast: $e")
end

## 7. Visualize Predictions and Historical Data

In [ ]:
try
    # Create visualization
    plot1 = plot(years, energy_values, 
        label="Historical Data", 
        linewidth=2, 
        marker=:circle,
        markersize=4,
        xlabel="Year",
        ylabel="Primary Energy Consumption",
        title="Italy Energy Consumption: Historical & Forecast",
        legend=:topright,
        size=(900, 500),
        dpi=100)
    
    # Add forecast line
    plot!(future_years, forecast_values,
        label="10-Year Forecast", 
        linewidth=2,
        marker=:square,
        markersize=4,
        linestyle=:dash,
        color=:red)
    
    # Add a vertical line to separate historical and forecast
    vline!([maximum(years) + 0.5], label="", linestyle=:dot, color=:gray, alpha=0.5)
    
    plot1
    
catch e
    println("Error in visualization: $e")
end

# Print summary statistics
println("\n=== ITALY ENERGY CONSUMPTION FORECAST SUMMARY ===")
println("Historical Period: $(minimum(years)) - $(maximum(years))")
println("Forecast Period: $(minimum(future_years)) - $(maximum(future_years))")
println("Average Historical Consumption: $(round(mean(energy_values), digits=2))")
if !isempty(forecast_values)
    println("Average Forecast Consumption: $(round(mean(forecast_values), digits=2))")
    println("Trend Direction: $(forecast_values[end] > energy_values[end] ? "INCREASING" : "DECREASING")")
end

## Summary

This notebook implements a complete machine learning pipeline for forecasting Italy's energy consumption:

✅ **Data Loading**: Downloaded OWID energy dataset and filtered for Italy  
✅ **Data Preprocessing**: Cleaned data and normalized values  
✅ **Model Training**: Trained multiple models (Linear, Ridge, Gaussian Process)  
✅ **10-Year Forecast**: Generated predictions for 2025-2034  
✅ **Visualization**: Created plots comparing historical vs. forecast data  

### Key Insights:
- Historical energy data provides a baseline for machine learning models
- Linear regression captures the overall trend in consumption patterns
- The 10-year forecast helps identify energy consumption trends for Italy
- Results can be used for energy planning and resource allocation

### Next Steps:
- Experiment with other features (GDP, population, renewable energy share)
- Try more advanced models (ARIMA, Prophet, Neural Networks)
- Add seasonal decomposition for better pattern capture
- Compare predictions with actual outcomes when data becomes available